# FantasyData Cleaning

In [10]:
# import packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [11]:
# Import all data
# assign path
path, dirs, files = next(os.walk("../data/raw_data/ppr_stats/"))
file_count = len(files)
# create empty list
dataframes_list = []
 
# append datasets to the list
for i in range(file_count):
    temp_df = pd.read_csv("../data/raw_data/ppr_stats/"+files[i], thousands=',')
    dataframes_list.append(temp_df)

In [12]:
# Array to store dataframe names
dataframes_names = []

for i in range(len(dataframes_list)):
    # If a player value is null, drop the row
    dataframes_list[i] = dataframes_list[i]
   
   # Rename all the dataframes as name of original csv file with date at the end (i.e. 2018_qb_stats.csv --> qb_stats_2018)
    globals()[files[i][5:].removesuffix('.csv')+'_'+files[i][:4]] = dataframes_list[i]
    dataframes_names.append(files[i][5:].removesuffix('.csv')+'_'+files[i][:4])

In [13]:
# Helpers to clean up resulting left merge
def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
    
def rename_x(df):
    for col in df:
        if col.endswith('_x'):
            df.rename(columns={col:col.rstrip('_x')}, inplace=True)

In [14]:
# Rename/Delete Columns, Fill null vals
for i in range(len(dataframes_names)):
    df1 = globals()[dataframes_names[i]]
    if i % 2 == 0:
        df1 = df1.rename(
            columns={'Name':'NAME', 'Age':'AGE', 'Position': 'POS','AverageDraftPositionPPR':"ADP"}).drop(
            columns={'Rank', 'Team', 'ByeWeek', 'PositionRank'}
            ).dropna(subset=['NAME']).fillna(0)
        df1 = df1.mask((df1) == '-', 0)
        df1 = df1.mask((df1) == 'BYE', 0)
        globals()[dataframes_names[i]] = df1
    else:
        df1 =  df1.rename(
            columns={'#':'RANK', 'Player':'NAME', 'Pos':'POS', 'TTL': 'TOTAL'}).drop(
            columns={'Team'}).dropna(
            subset=['NAME']).fillna(0)   
             
        df1 = df1.mask((df1) == '-', 0)
        df1 = df1.mask((df1) == 'BYE', 0)
        # Convert all objects to floats, Rename Columns
        cols = df1.columns
        week_cols = df1.columns[~df1.columns.isin(['RANK','NAME', 'POS', 'AVG','TOTAL'])]
        
        df1[cols[3:]] = df1[cols[3:]].apply(pd.to_numeric, errors ='coerce')
        df1.rename(columns = dict(zip(week_cols, 'WEEK_' + week_cols)), inplace=True)
        globals()[dataframes_names[i]] = df1

In [15]:
def merge(ppr_adp_2023, df2):
        # # merge stats and advanced stats
        df3= pd.merge(left=ppr_adp_2023, right=df2, on='NAME').fillna(0)
        drop_y(df3)
        rename_x(df3)
        
        df3 = df3[(df3['POS'] == 'QB') | (df3['POS'] == 'RB') | (df3['POS'] == 'WR') | (df3['POS'] == 'TE')]
        
        return df3

In [16]:
# 2023 ADP
df1 = pd.read_csv('../data/raw_data/ppr_stats/2023_ppr_adp.csv')
ppr_adp_2023 = df1.drop(columns=['Team', 'ByeWeek', 'PositionRank']).rename(
    columns={'Rank': 'RANK', 'Name':'NAME', 
             'Age':'AGE', 'Position': 'POS','AverageDraftPositionPPR':"ADP"}
)

In [17]:
# export new csv files
for df in dataframes_names:
    globals()[df].to_csv(f'../data/clean_data/ppr_stats_clean/{df}_clean.csv', index=False)
    
ppr_adp_2023.to_csv('../data/clean_data/ppr_stats_clean/ppr_adp_2023_clean.csv', index=False)